Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

# CSCI 3155 Assignment 4 : Inductive Definitions and Case Pattern Matching.

This assignment asks you to write scala programs. 

**Restrictions** apply to each problem in terms of forbidden Scala features and API functions. Please read them carefully and ask for clarifications from the course staff over Piazza or during office hours if unsure.

### Assignment-wide Restrictions

These restrictions apply to the entire assignment.
  - No loops can be used.
  - No vars can be used.
  - No return keyword.

Note: `???` indicates that there is a missing function or code fragment that needs to be filled in. In scala, 
it is also a macro that throws a `NotImplemented` exception. Make sure that you remove the `???` and replace it with the answer. 

Use the test cases provided to test them. You are also encouraged to write your own test cases to help debug your work. However, please delete any extra cells you may have created lest they break our autograder.

**Very Important:** 
 - Please run the cell that defines the functions `passed` and `testWithMessage` below whenever you restart the notebook.
 - Before you submit the assignment, run `kernel -> Restart & Run All` to check that all your tests pass.

### Connor O'Reilly

In [1]:
// TEST HELPER

// FIRST RUN THIS CELL EVERY TIME YOU START THE NOTEBOOK
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

def testWithMessage[T](v1: T, expected: T, testID: String) = { 
    println(s"Test $testID"); 
    println(s"\t Expected: $expected, your code returned: $v1")
    assert (v1 == expected, s"Test $testID FAILED.")
    println("\t Passed!")
}


defined function passed
defined function testWithMessage

## Problem 1: Manipulating ASTs, Inference Rules (30 points)

### 1A: Derivatives of Expressions (25 points)

We have defined a grammar for arithmetic expressions in our class notes. 
$$\newcommand\Expr{\mathbf{Expr}}$$

$$\begin{array}{rcl}
\Expr & \Rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{String}) \\ 
& | & Plus (\Expr, \Expr) \\
& | & Minus (\Expr, \Expr) \\
& | & Mult( \Expr, \Expr) \\
& |& Div(\Expr, \Expr) \\
& | & Exp(\Expr) \\
& |& Sine(\Expr) \\
& | & Cosine(\Expr) \\
\end{array}$$


For this problem, you will be writing an _automatic differentiation_ method that, given an expression `e` which involves just a single identifier `x` (no need to check this fact), will return an expression for `de/dx`, the derivative of `e` with respect to `x`.

`derivative` takes as input an expression ($\Expr$) and evaluates to an expression ($\Expr$).

### Example

We know that $\frac{d \sin(2x)}{dx} = 2 \cos( 2x) $. Therefore, 

- Input:  `e = Sine(Mult(Ident("x"), Const(2.0)))` 
  - Output: `Mult(Const(2.0), Cosine(  Mult(Ident("x"), Const(2.0)))`.  


We will write down the inference rules for derivative, as follows.

A rule for constants ($\frac{dc}{dx} = 0, c \in \mathbb{R}$)

$\begin{array}{c}
\\
\hline 
\text{derivative}( \texttt{Const(f)} , x) = \texttt{Const(0.0)} \\
\end{array} \mathbf{(Constant)}$      

A rule for identifiers $\frac{dx}{dx} = 1, \frac{dy}{dx} = 0$ for $y \not= x$.

$\begin{array}{c}
\\
\hline 
\text{derivative}( \texttt{Ident(s)} , x) = \left\{ \begin{array}{ll} \texttt{Const(1.0)} & x = s \\
\texttt{Const(0.0)} & \text{otherwise} \end{array} \right.\\
\end{array} \mathbf{(Identifier)}  \;\;\;
$

A rule for plus $\frac{d}{dx} (e_1 + e_2) = \frac{de_1}{dx} + \frac{de_2}{dx}$.

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Plus(e1, e2)}, x) = \texttt{Plus(f1, f2)} \\
\end{array} \mathbf{(Plus)} $

A rule for multiplication: $\frac{d}{dx} (e_1 e_2) = e_2 \frac{de_1}{dx} + e_1 \frac{de_2}{dx}$.

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Mult(e1, e2)}, x) = \texttt{Plus(Mult(f1, e2), Mult(f2, e1))} \\
\end{array} \mathbf{(Mult)} $

A rule for division $\frac{d}{dx} \left(\frac{e_1}{e_2}\right) = \frac{\frac{de_1}{dx}}{e_2} - \frac{e_1 \frac{d e_2}{dx}}{e_2^2}$

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Div(e1, e2)}, x) = \texttt{Minus(Div(f1, e2), Div(Mult(e1, f2), Mult(e2, e2)))} \\
\end{array} \mathbf{(Div)} $

A rule for exponentiation $\frac{d}{dx} \left(e^{e_1}\right) = e^{e_1} \frac{de_1}{dx}$

$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1}\\
\hline
\text{derivative}(\texttt{Exp(e1)}, x) = \texttt{Mult(Exp(e1), f1)} \\
\end{array} \mathbf{(Exp)} $

The rules for $\mathbf{Minus}$, $\mathbf{Sine}$, and $\mathbf{Cosine}$ are left for you to write.
You do not need to write your solution for these rules in this notebook, __but__, you do need to give them in code below.
(Don't forget about the chain rule!)

In [2]:
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr
case class Exp(e: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Sine
defined class Cosine
defined class Exp

Write a function `derivativeExpr` that calculates the derivative of an expression
`e` w.r.t a given identifier as a string `x`.

  - No loops can be used.
  - No vars can be used.
  - No return keyword.


In [3]:
def derivativeExpr(e: Expr, x: String): Expr = e match{
    case Const(f) => Const(0.0)  
    case Ident(f) => {
           if(f.contains(x)){
                Const(1.0)
           }else{
               Const(0.0)
           }
    }
    case Plus(e1, e2) => {
        val f1 = derivativeExpr(e1,x)
        val f2 = derivativeExpr(e2,x)
        Plus(f1,f2)
    }
    case Minus(e1, e2) => {
        val f1 = derivativeExpr(e1,x)
        val f2 = derivativeExpr(e2,x)
        Minus(f1,f2)
    }
    case Mult(e1,e2) => {
        val f1 = derivativeExpr(e1,x)
        val f2 = derivativeExpr(e2,x)
        Plus(Mult(f1,e1),Mult(f1,e1))
    }
    case Div(e1,e2) => {
        val f1 = derivativeExpr(e1,x)
        val f2 = derivativeExpr(e2,x)
        Minus(Div(f1,e2), Div(Mult(e1, f2), Mult(e2,e2)))
    }
    case Exp(e1) => {
        val f1 = derivativeExpr(e1,x)
        Mult(Exp(e1), f1)
    }
    case Sine(e1) => {
        Cosine(e1)
    }
    case Cosine(e1) => {
        Minus(Const(0.0), Sine(e1))
    }

}

defined function derivativeExpr

In [4]:
// TEST HELPERS
def evalExpr (e: Expr, env: Map[String, Double]): Double = e match {
    case Const (f) => f
    case Ident (str) => { if (env.contains(str)){
                                env(str)
                            } else {
        throw new IllegalArgumentException(s"Environment does not contain mapping for $str")
    }
                        }
    case Plus(e1, e2) => {
        (evalExpr(e1, env)) + (evalExpr(e2, env))
    }
    
    case Minus(e1, e2) => {
        (evalExpr(e1, env)) - (evalExpr(e2, env))
    }
    
    case Mult(e1, e2) => {
        (evalExpr(e1, env)) * (evalExpr(e2, env))
    }
    
    case Div(e1, e2) => {
        val v2 = evalExpr(e2, env)
        if (math.abs(v2) > 1E-09){
            (evalExpr(e1, env)) / (v2)
        } else {
            throw new IllegalArgumentException("Division by Zero Error -bailing out")
        }
    }
    
    case Exp(e) => math.exp( evalExpr(e, env))
    
    case Sine(e) => math.sin( evalExpr(e, env))
    
    case Cosine(e) => math.cos(evalExpr(e, env))
}

def testExpressions(exp: Expr, deriv_expected: Expr, testVals: List[Double]): Boolean = {
    val tol: Double = 1E-06
    val deriv_act = derivativeExpr(exp, "x")
    testVals forall { 
            x => {
              val res = math.abs( evalExpr(deriv_act, Map("x"-> x)) - evalExpr(deriv_expected, Map("x" -> x)) ) <= tol
              if (!res) { println(s"Failed at $x")}
              res
            }
    }
}

val allVals = List(-5.0, -4.5, -4.0, -3.5, -3.0, -2.5, -1.9, -1.4, -1.0, -0.5, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0)

defined function evalExpr
defined function testExpressions
allVals: List[Double] = List(
  -5.0,
  -4.5,
  -4.0,
  -3.5,
  -3.0,
  -2.5,
  -1.9,
  -1.4,
  -1.0,
  -0.5,
  0.1,
  0.5,
  1.0,
  1.5,
  2.0,
  2.5,
  3.0,
  3.5,
  4.0,
  4.5,
  5.0
)

In [5]:
// BEGIN TEST
// Please make sure you run the cell that defines the function 
// testExpressions above. 
val e1 = Plus(Ident("x"), Const(2.0)) // derivative of x + 2
assert(testExpressions(e1, Const(1.0), allVals ), s"Test 1 Failed -- Input: $e1 ")

passed(5)
// END TEST


*** Tests Passed (5 points) ***


e1: Plus = Plus(Ident("x"), Const(2.0))

In [6]:
// BEGIN TEST
// Please make sure you have already run the cell
// that defines the function  testExpressions above. 
val e2 = Plus(Cosine(Ident("x")), Sine(Ident("x")))
val ed2 = Minus(Cosine(Ident("x")), Sine(Ident("x")))
assert(testExpressions(e2, ed2, allVals), s"Test 2 Failed: Input is $e2")
passed(5)
// END TEST


*** Tests Passed (5 points) ***


e2: Plus = Plus(Cosine(Ident("x")), Sine(Ident("x")))
ed2: Minus = Minus(Cosine(Ident("x")), Sine(Ident("x")))

In [7]:
// BEGIN TEST
// Please make sure you have already run the cell
// that defines the function  testExpressions above. 
val x = Ident("x")
val e3 = Exp(Mult(x, x))
val ed3 = Mult(Mult(Const(2.0), x), e3)
assert(testExpressions(e3, ed3, allVals), s"Test 3 Failed: Input is $e3")

passed(5)
// END TEST


*** Tests Passed (5 points) ***


x: Ident = Ident("x")
e3: Exp = Exp(Mult(Ident("x"), Ident("x")))
ed3: Mult = Mult(
  Mult(Const(2.0), Ident("x")),
  Exp(Mult(Ident("x"), Ident("x")))
)

In [8]:
// BEGIN TEST
// Please make sure you have already run the cell
// that defines the function  testExpressions above. 
val e4 = Div(x, Plus(x, Const(2.0)))
val ed4 = Div(Const(2.0), Mult(Plus(x, Const(2.0)), Plus(x, Const(2.0))) )
assert(testExpressions(e4, ed4, allVals), s"Test 4 Failed: Input is $e4")

val e5 = Sine(Mult(Exp(Minus( Cosine(Div(x,x)), Cosine(Const(1.0)) )), x))
val ed5 = Cosine(x)
assert(testExpressions(e5, ed5, allVals), s"Test 5 Failed: Input is $e5")

passed(10)
// END TEST


*** Tests Passed (10 points) ***


e4: Div = Div(Ident("x"), Plus(Ident("x"), Const(2.0)))
ed4: Div = Div(
  Const(2.0),
  Mult(Plus(Ident("x"), Const(2.0)), Plus(Ident("x"), Const(2.0)))
)
e5: Sine = Sine(
  Mult(
    Exp(Minus(Cosine(Div(Ident("x"), Ident("x"))), Cosine(Const(1.0)))),
    Ident("x")
  )
)
ed5: Cosine = Cosine(Ident("x"))

### 1B: Constant Folding Expressions (20 points)

Constant folding is an operation that compilers often do to simplify (parts of) expressions that evaluate to constants. For instance, suppose one is given 
an expression of the form  ` 2 * 3 +  x `, represented as
`Plus( Mult( Const(2), Const(3)), Ident(x)) `, we could directly simplify it to `6 + x` or `Plus( Const(6), Ident(x))`. In other words, we evaluate parts of an expression without needing to know the value of any of the identifiers.

We will carry out constant folding according to some semantic rules that we will write below.

Constant folding a constant does not change it.

$$\begin{array}{c}
\\\hline
\mathsf{constFold}(\texttt{Const(f)}) = \texttt{Const(f)}\\ 
\end{array}\;\; \mathsf{(const-fold)}$$

Constant folding an identifier does not change it.

$$\begin{array}{c}
\\\hline
\mathsf{constFold}(\texttt{Ident(x)}) = \texttt{Ident(x)}\\ 
\end{array}\;\; \mathsf{(ident-fold)}$$

Constant folding a plus when both operands fold into constants.

$$\begin{array}{c}
\mathsf{constFold}(\texttt{e1}) = \texttt{Const}(f1),\ \ \mathsf{constFold}(\texttt{e2}) = \texttt{Const}(f2),\ \ 
\\\hline
\mathsf{constFold}(\texttt{Plus(e1, e2)}) = \texttt{Const(f1+ f2)}\\ 
\end{array}\;\; \mathsf{(plus-const-fold-1)}$$

Constant folding a plus when one of the operands does not fold into constants.

$$\begin{array}{c}
\mathsf{constFold}(\texttt{e1}) = \texttt{t1},\ \ \mathsf{constFold}(\texttt{e2}) = \texttt{t2},\ \ \text{at least one of}\ \texttt{t1}\ or\ \texttt{t2}\  \text{is not Const} \\ 
\\\hline
\mathsf{constFold}(\texttt{Plus(e1, e2)}) = \texttt{Plus(t1, t2)}\\ 
\end{array}\;\; \mathsf{(plus-const-fold-2)}$$

Similar rules apply (with a few modifications) for constant folding other cases `Minus`, `Div`, `Mult`. Do not worry about division by zero for this assignment.

__Also, there is no need to handle Sine/Cosine/Exp__ for this assignment. 
Throw an `IllegalArgumentException` if your expression has those functions in it. 

Write a function `constFold` that takes in an `Expr` and returns an expression after constant folding.
  - No loops can be used.
  - No vars can be used.
  - No return keyword.


In [9]:
def constFold(e: Expr): Expr = e match{
    case Const(e1) => Const(e1)
    case Ident(e1) => Ident(e1)
    case Plus(e1, e2) => {
        val idk1 = constFold(e1)
        val idk2 = constFold(e2)
        //check cases
        idk1 match {
            
            case Const(dob1) => { 
                //if the first constFold turns out to be a constant
                idk2 match{
                    case Const(dob2) => {Const(dob1+dob2)} // if second case turns out to be a constant
                    case _ => {Plus(Const(dob1),idk2)}  // if second case is not a constant
                }
            
            }
            case _ => { Plus(idk1,idk2) } // if first case is not a constant
            
        }
    
    }
    case Minus(e1, e2) => {
        val idk1 = constFold(e1)
        val idk2 = constFold(e2)
        //check cases
        idk1 match {
            
            case Const(dob1) => { 
                //if the first constFold turns out to be a constant
                idk2 match{
                    case Const(dob2) => {Const(dob1-dob2)} // if second case turns out to be a constant
                    case _ => {Minus(Const(dob1),idk2)}  // if second case is not a constant
                }
            
            }
            case _ => { Minus(idk1,idk2) } // if first case is not a constant
            
        }
    
    }
    case Div(e1, e2) => {
        val idk1 = constFold(e1)
        val idk2 = constFold(e2)
        //check cases
        idk1 match {
            
            case Const(dob1) => { 
                //if the first constFold turns out to be a constant
                idk2 match{
                    case Const(dob2) => {Const(dob1/dob2)} // if second case turns out to be a constant
                    case _ => {Div(Const(dob1),idk2)}  // if second case is not a constant
                }
            
            }
            case _ => { Div(idk1,idk2) } // if first case is not a constant
            
        }
    
    }
    case Mult(e1, e2) => {
        val idk1 = constFold(e1)
        val idk2 = constFold(e2)
        //check cases
        idk1 match {
            
            case Const(dob1) => { 
                //if the first constFold turns out to be a constant
                idk2 match{
                    case Const(dob2) => {Const(dob1*dob2)} // if second case turns out to be a constant
                    case _ => {Mult(Const(dob1),idk2)}  // if second case is not a constant
                }
            
            }
            case _ => { Mult(idk1,idk2) } // if first case is not a constant
            
        }
    
    }
    case Sine(e1) => throw new IllegalArgumentException("Expression contains Sine(Expr)")
    case Cosine(e1) => throw new IllegalArgumentException("Expression contains Cosine(Expr)")
    case Exp(e1) => throw new IllegalArgumentException("Expression contains Exp(Expr)")
    
    case _ => throw new IllegalArgumentException("Unhandled case")
    
}

defined function constFold

In [10]:
import scala.language.implicitConversions
implicit def toExpr(f: Double) = Const(f)
implicit def toExpr(f: Int) = Const(f.toDouble)
implicit def toExpr(s: String) = Ident(s)
def pl(e1: Expr, e2: Expr) = Plus(e1, e2)
def ms(e1: Expr, e2: Expr) = Minus(e1, e2)
def st(e1: Expr, e2: Expr) = Mult(e1, e2)
def dv(e1: Expr, e2: Expr) = Div(e1, e2)

import scala.language.implicitConversions

defined function toExpr
defined function toExpr
defined function toExpr
defined function pl
defined function ms
defined function st
defined function dv

In [11]:
val e = st(pl(3, 3), dv(3, 2))
println(e)
testWithMessage(constFold(e), Const(9.0), "#1")
passed(5)

Mult(Plus(Const(3.0),Const(3.0)),Div(Const(3.0),Const(2.0)))
Test #1
	 Expected: Const(9.0), your code returned: Const(9.0)
	 Passed!

*** Tests Passed (5 points) ***


e: Mult = Mult(Plus(Const(3.0), Const(3.0)), Div(Const(3.0), Const(2.0)))

In [12]:
val e = ms(st(2, 3), pl(3, 2))
println(e)
testWithMessage(constFold(e), Const(1.0), "#2")
passed(5)

Minus(Mult(Const(2.0),Const(3.0)),Plus(Const(3.0),Const(2.0)))
Test #2
	 Expected: Const(1.0), your code returned: Const(1.0)
	 Passed!

*** Tests Passed (5 points) ***


e: Minus = Minus(Mult(Const(2.0), Const(3.0)), Plus(Const(3.0), Const(2.0)))

In [13]:
val e = ms(st(2, "x") , pl(3, 2))
println(e)
testWithMessage(constFold(e), Minus( Mult(Const(2.0), Ident("x")), Const(5.0)), "#3")
passed(5)

Minus(Mult(Const(2.0),Ident(x)),Plus(Const(3.0),Const(2.0)))
Test #3
	 Expected: Minus(Mult(Const(2.0),Ident(x)),Const(5.0)), your code returned: Minus(Mult(Const(2.0),Ident(x)),Const(5.0))
	 Passed!

*** Tests Passed (5 points) ***


e: Minus = Minus(Mult(Const(2.0), Ident("x")), Plus(Const(3.0), Const(2.0)))

In [14]:
val e = st(dv("x", 3) , ms(st(3, 2), "y"))
println(e)
testWithMessage(constFold(e), Mult( Div(Ident("x"),Const(3.0)), Minus(Const(6.0), Ident("y"))), "#3")
passed(5)

Mult(Div(Ident(x),Const(3.0)),Minus(Mult(Const(3.0),Const(2.0)),Ident(y)))
Test #3
	 Expected: Mult(Div(Ident(x),Const(3.0)),Minus(Const(6.0),Ident(y))), your code returned: Mult(Div(Ident(x),Const(3.0)),Minus(Const(6.0),Ident(y)))
	 Passed!

*** Tests Passed (5 points) ***


e: Mult = Mult(
  Div(Ident("x"), Const(3.0)),
  Minus(Mult(Const(3.0), Const(2.0)), Ident("y"))
)

## Problem 2 

### Part (A)  - 5 points

Write a function using case pattern matching whether a given list of numbers has the `fibonacci` property: i.e, every element of the list (other than the first two) is the sum of two previous elements. 

Lists of length less than or equal to two are naturally fibonacci lists.

 - Do not use any list API functions other than length.
 - Use case pattern matching over scala lists.
 - Function must be tail recursive.
 - No loops can be used.
 - No vars can be used.
 - No return keyword.



In [15]:
import scala.annotation.tailrec
@tailrec
def isFibonacciList(lst: List[Int], param: Boolean = true): Boolean = lst match{
    
    case Nil => param //empty list case
    case lst if(lst.length < 3 ) => param//list with one or two elements
    //past previous case, list contains more than two elements
    case a::b::c::tail => {
            if( (a + b) == c ) {
                isFibonacciList(b::c::tail,true)
            }
            else{
                false
            }
            
        }
    case _ => throw new IllegalArgumentException("Unhandled case") 
}
    


import scala.annotation.tailrec

defined function isFibonacciList

In [16]:
testWithMessage(isFibonacciList(List(1,2,3,5)), true, "isFibonacciList([1,2,3])")
testWithMessage(isFibonacciList(List(-20, -15)), true, "isFibonacciList([-20, -15])")
testWithMessage(isFibonacciList(Nil), true, "isFibonacciList([])")
testWithMessage(isFibonacciList(List(1,2, 3, 5, 8, 13, 21)), true, 
               "isFibonacciList([1,2,3,5,8,13,21])")
testWithMessage(isFibonacciList(List(-1,1, 0,  0, 1, 2, 3)), false, 
               "isFibonacciList([-1, 1, 0, 0, 1, 2, 3])")
testWithMessage(isFibonacciList(List(-1,1, 0,  1, 1, 2, 1)), false, 
               "isFibonacciList([-1, 1, 0, 1, 1, 2, 1])")

passed(5)

Test isFibonacciList([1,2,3])
	 Expected: true, your code returned: true
	 Passed!
Test isFibonacciList([-20, -15])
	 Expected: true, your code returned: true
	 Passed!
Test isFibonacciList([])
	 Expected: true, your code returned: true
	 Passed!
Test isFibonacciList([1,2,3,5,8,13,21])
	 Expected: true, your code returned: true
	 Passed!
Test isFibonacciList([-1, 1, 0, 0, 1, 2, 3])
	 Expected: false, your code returned: false
	 Passed!
Test isFibonacciList([-1, 1, 0, 1, 1, 2, 1])
	 Expected: false, your code returned: false
	 Passed!

*** Tests Passed (5 points) ***


### Part (B) - 5 points

Write a tail recursive function that returns the length of the __longest ascending prefix__ of a list. Ie., given a list
`List(a0, a1, a2, ..., aj, ..., an-1)`,  it returns the first index
`j` wherein `aj > aj+1`. If no such `j` exists, then it returns `n-1`.

- No use of any list API method allowed.
- Use case pattern matching.
- Make your function tail recursive.
- No loops can be used.
- No vars can be used.
- No return keyword.


#### Examples

- `longestAscendingPrefix(List(1, 2, 3, 4, 3)) = 3`
- `longestAscendingPrefix(List(5, 2, 3, 4, 3)) = 0`
- `longestAscendingPrefix(List()) = 0`
- `longestAscendingPrefix(List(1, 2, 3, 4, 10)) = 4`






In [17]:
import scala.annotation.tailrec
@tailrec
def longestAscendingPrefix(lst: List[Int], acc: Int = 0): Int = lst match{
    case Nil => 0
    case a::b::tail => {
        // if at end of list
        if(tail == Nil){
            //tail is empty just check a and b
            if(a > b){
                acc
            } else {
                acc + 1
            }
        }else{
            //tail is not empty
            if(a > b){
                acc
            } else{
                val acc1 = acc + 1
                longestAscendingPrefix(b::tail,acc1)
            }
        }
    }
    case _ => throw new IllegalArgumentException("Unhandled case")
    
}

import scala.annotation.tailrec

defined function longestAscendingPrefix

In [18]:
testWithMessage(longestAscendingPrefix(List(1, 2, 3, 4, 3)), 3, "#1")
testWithMessage(longestAscendingPrefix(List(5, 2, 3, 4, 3)), 0, "#2")
testWithMessage(longestAscendingPrefix(List()) , 0, "#3")
testWithMessage(longestAscendingPrefix(List(1, 2, 3, 4, 10)), 4, "#4")
passed(5)

Test #1
	 Expected: 3, your code returned: 3
	 Passed!
Test #2
	 Expected: 0, your code returned: 0
	 Passed!
Test #3
	 Expected: 0, your code returned: 0
	 Passed!
Test #4
	 Expected: 4, your code returned: 4
	 Passed!

*** Tests Passed (5 points) ***


## That's All Folks